In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime as dt
import sys
sys.path.append('/Users/dnowacki/Documents/Grand Bay/py')
sys.path.append('/Users/dnowacki/Documents/python')
sys.path.append('/Users/dnowacki/Documents/stglib')
import stglib
import gbts
import shutil
import plotly
import plotly.graph_objs as go
import pandas as pd
import xarray as xr
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
gndcrmet = gbts.read_nerrs('/Volumes/Backstaff/field/gb/gndnerr/gndcrmet/GNDCRMET.csv')
basedir = '/Volumes/Backstaff/field/gb_proc/'

In [ ]:
def load_raw(site, dep, basedir, waves=False):
    fildir = basedir + site + dep + '/' + site + '1' +  dep.upper() + 'aqd'  + '/'

    RAW = {}
    
    if waves == False:
        print(fildir + site + '1' + dep.upper() + 'aqd-raw.cdf')
        RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqd-raw.cdf', autoclose=True)
    elif waves == True:
        print(fildir + site + '1' + dep.upper() + 'aqdwvs-raw.cdf')
        RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqdwvs-raw.cdf', autoclose=True)
    
    return RAW, fildir

def proc_atmpres(site, dep, offset, basedir, waves=False):
    RAW, fildir = load_raw(site, dep, basedir, waves=waves)

    met = gndcrmet['BP']/100
    met = met.rename('atmpres')
    met = met.reindex(time=RAW['time'], copy=True, method='nearest')
    met.attrs.update(offset=offset)
    
    if waves == False:
        print('in proc nowaves')
        met.to_netcdf(fildir + 'atmpres.cdf')
    elif waves == True:
        print('in proc waves')
        met.to_netcdf(fildir + 'atmpres-wvs.cdf')

    return RAW 

def load_clean(site, dep, basedir):
    fildir = basedir + site + dep + '/' + site + '1' +  dep.upper() + 'aqd'  + '/'

    RAW = {}
    print(fildir + site + '1' + dep.upper() + 'aqd-a.nc')
    RAW = xr.open_dataset(fildir + site + '1' + dep.upper() + 'aqd-a.nc', decode_times=False, autoclose=True)
    RAW['time'] = RAW['time_cf']
    RAW = RAW.drop('time2')
    RAW = xr.decode_cf(RAW)
    
    return RAW

In [ ]:
# set offset for 1076b & 1078b to 10.15 to equalize postitive, negative anomalies for 
# atmos-corrected pressure at beginning/end of deployment
print('Processing Aquadopp currents')
for site, dep, offset in zip(['1076', '1076', '1077', '1077', '1078', '1078', '1079', '1079'], 
                             ['a',    'b',    'a',    'b',    'a',    'b',    'a',    'b'], 
                             [-10.15,  -10.25, -10.20, -10.20, -10.15, -10.25,  -10.08,  -9.95]):
    # 1077a offset is an estimate, no good pressure data outside of in-water time
    proc_atmpres(site, dep, offset, basedir, waves=False)

print('Processing Aquadopp waves')
for site, dep, offset in zip(['1076', '1076', '1078', '1078', '1079'], 
                             ['a',    'b',    'a',    'b',    'b'], 
                             [-10.15,  -10.25, -10.15, -10.25,  -9.95]):
    
    proc_atmpres(site, dep, offset, basedir, waves=True)

In [ ]:
print(RAW['Pressure'])

In [ ]:
site = '1076'
dep = 'b'
offset = 10.15
RAW, fildir = load_raw(site, dep, basedir)
VEL = load_clean(site, dep, basedir)

In [ ]:
trace = go.Scatter(
    x = RAW['time'].values,
    y = RAW['Pressure'].values, 
    name = 'Raw pressure'
)

trace2 = go.Scatter(
    x = VEL['time'].values,
    y = VEL['P_1ac'].values,
    name = 'Atmos. corrected'
)
layout = go.Layout(title=site + dep + ' offset: ' + str(offset))
data = [trace, trace2]
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)